# Multi Latent Attention(MLA)

MHA/GQA 网络中在 Inference 时会产生大量的 KV-Cache，其占用越多，将减少 Decoding 时 batching 数量。

MLA 是为了减少 KV-Cache 而生的技术。 为了快速理解 MLA 是一种类似 MQA(Multi_Query_Attention) 的技术（即单头KV），MLA做到了

1. MLA 达到 MQA 单头 KV 的 cache 量级， 对于 K、V 需要存两份数据， MLA 只要存一份 Cache 数据
2. 将 MQA 的 KV 投影看成是一种特征压缩（40头->1头， 减少97.5% 数据量），其算 MHA 是 KV 单头share成多头
3. MLA 从单头到多头的映射是通过一个升维投影矩阵得到的， 对于 MQA 类压缩投影，被 MLA 视为是将 latent特征 投影到 隐空间(learnable）

MQA 公式化为：

$$
k_\text{mqa} = WK_\text{down}(X), X \in \mathbb{R} ^{N \times d}, W_\text{down} \in \mathbb{R} ^{d  \times d'},  
$$

$$
K \gets \text{extend}(k_\text{mqa})
$$

MLA 公式化为

$$
c = W_\text{down}(X), X \in \mathbb{R} ^{N  \times  d}, W_\text{down} \in \mathbb{R} ^{d  \times d'},
$$
$$
K = WK_\text{up}(c)， WK_\text{up} \in \mathbb{R} ^{d'  \times d},  
$$

MLA 额外增加一些工程 trick，保证精度、RoPE适配等问题。 


为什么 MLA 效果比 MQA 好？本质上：

1. pretrained 任务上学习压缩/低秩表示，本身高维特征有冗余和稀疏的特性，这是能压缩的基础
2. LoRA 技术证明，特征的低秩表示的有效性，除了压缩外，还存在升维投影，这是 MQA 不具备的
3. MLA 与 MHA 是否等效？ 可以认为 MLA 的投影矩阵做了一次矩阵分解 Wq = WAWB 去 近似满头 Wq = [dim,dim]

# setting

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
torch.manual_seed(42)

In [2]:
# @dataclass
class ModelArgs:
    dim: int = 64
    n_heads: int = 8
    n_kv_heads: int =  2

    # down 两者远小于 dim
    dc_kv: int = 4
    dc_q: int = 4

bs = 3
seq_len = 5

config = ModelArgs()
print(config)

## Multi Query Attention

In [3]:
class MultiQueryAttention(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.n_heads = args.n_heads
        self.dim = args.dim
        self.head_dim = args.dim // args.n_heads # 18/6 = 3

        self.wq = nn.Linear(self.dim, self.dim, bias=False)
        self.wk = nn.Linear(self.dim, self.head_dim, bias=False)
        self.wv = nn.Linear(self.dim, self.head_dim, bias=False)
        self.wo = nn.Linear(self.dim, self.dim, bias=False)
        
    def forward(
        self,
        x: torch.Tensor,
    ):
        bsz, seqlen, _ = x.shape
        Q, K, V = self.wq(x), self.wk(x), self.wv(x)

        # 多头Q
        Q = Q.view(bsz, seqlen, self.n_heads, self.head_dim)
        Q = Q.transpose(1, 2)  # (bs, n_heads, seqlen, head_dim)
        
        # 单头KV
        K = K[:, None, :, :]
        V = V[:, None, :, :]
        print(Q.shape, K.shape)
    
        S = Q @ K.transpose(2, 3) / math.sqrt(self.head_dim)
        P = F.softmax(S.float(), dim=-1)
        Z = P @ V 
        Z = Z.transpose(1, 2).contiguous().view(bsz, seqlen, -1)
        O = self.wo(Z)
        return O

In [4]:
MQA = MultiQueryAttention(config)
print(MQA)

MultiQueryAttention(
  (wq): Linear(in_features=64, out_features=64, bias=False)
  (wk): Linear(in_features=64, out_features=8, bias=False)
  (wv): Linear(in_features=64, out_features=8, bias=False)
  (wo): Linear(in_features=64, out_features=64, bias=False)
)


In [5]:
X = torch.randn(bs, seq_len, config.dim)

In [6]:
O = MQA(X)
print(X.shape)
print(O.shape)

torch.Size([3, 8, 5, 8]) torch.Size([3, 1, 5, 8])
torch.Size([3, 5, 64])
torch.Size([3, 5, 64])


## Multi-Heads Latent Attention

### model

1. 以下用down和up权重矩阵，代替直接的wq,wk,wv
2. MLA矩阵发生于训练之时

In [7]:
class MultiHeadsLatentAttention(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.n_heads = args.n_heads
        self.dim = args.dim
        self.head_dim = args.dim // args.n_heads # 18/6 = 3
        self.dc_kv = args.dc_kv # down_c kv
        self.dc_q = args.dc_q # down_c kv

        # Q
        self.wq_down = nn.Linear(self.dim, args.dc_q, bias=False,)
        self.wq_up = nn.Linear(args.dc_q, self.dim , bias=False,)

        # 单个 C 映射到 K、V
        self.wkv_down = nn.Linear(self.dim, args.dc_kv, bias=False,)
        self.wk_up = nn.Linear(args.dc_kv, self.dim, bias=False,)
        self.wv_up = nn.Linear(args.dc_kv, self.dim, bias=False,)
        
        self.wo = nn.Linear(self.dim, self.dim, bias=False,)

In [8]:
print(config.dc_kv)
print(config.dc_q)
mla = MultiHeadsLatentAttention(config)
print(mla)

4
4
MultiHeadsLatentAttention(
  (wq_down): Linear(in_features=64, out_features=4, bias=False)
  (wq_up): Linear(in_features=4, out_features=64, bias=False)
  (wkv_down): Linear(in_features=64, out_features=4, bias=False)
  (wk_up): Linear(in_features=4, out_features=64, bias=False)
  (wv_up): Linear(in_features=4, out_features=64, bias=False)
  (wo): Linear(in_features=64, out_features=64, bias=False)
)


### forward

In [9]:
X = torch.randn(bs, seq_len, config.dim)

In [10]:
# 该段代码是关键，先降维，再升维
C_Q = mla.wq_down(X)
Q = mla.wq_up(C_Q)

C = mla.wkv_down(X)
K = mla.wk_up(C)
V = mla.wv_up(C)

print(C_Q.shape, C.shape) # latent dim
print(Q.shape)
print(K.shape)
print(V.shape)

torch.Size([3, 5, 4]) torch.Size([3, 5, 4])
torch.Size([3, 5, 64])
torch.Size([3, 5, 64])
torch.Size([3, 5, 64])


In [11]:
# 与传统多头注意力无差别
Q = Q.reshape(bs, seq_len, mla.n_heads, mla.head_dim)
K = K.reshape(bs, seq_len, mla.n_heads, mla.head_dim)
V = V.reshape(bs, seq_len, mla.n_heads, mla.head_dim)

Q = Q.transpose(1,2)
K = K.transpose(1,2) # bs, n_heads, [seq_len, head_dim]
V = V.transpose(1,2)

S = Q @ K.transpose(2, 3) / math.sqrt(mla.head_dim)
P = F.softmax(S.float(), dim=-1)
Z = P @ V 
Z = Z.transpose(1, 2).contiguous().view(bs, seq_len, -1)
O = mla.wo(Z)
print(O.shape)

torch.Size([3, 5, 64])


### 矩阵吸收

以上参数发生在训练之时，训练完成后，我们可以做吸收操作，具体指wq参数和Wo参数，

我们写出如下等式：

Q = wq_up * ( wq_down * h ) = (wq_up * wq_down) * h

Q = wq * h 

目的是什么？

1. 训练时省显存
2. 训练完，推理Q满矩阵保精度，
3. 由于KV Cache的存在，decoding阶段时one-by-one token进行q计算
4. KV Cache极具减少存储体现。

In [12]:
WQ = (mla.wq_up.weight.data @ mla.wq_down.weight.data).t() 

C_Q = mla.wq_down(X)
Q = mla.wq_up(C_Q)
print(Q[0,0,:5])

Q = X @ WQ
print(Q[0,0,:5])

tensor([-1.0488,  0.7823, -0.7392,  0.9954, -0.0828], grad_fn=<SliceBackward0>)
tensor([-1.0488,  0.7823, -0.7392,  0.9954, -0.0828])


### 矩阵吸收后的forward(非训练阶段）

In [13]:
Q = X @ WQ
C = mla.wkv_down(X)
K = mla.wk_up(C)
V = mla.wv_up(C)

# do attention

### KV cache存的是什么？

传统MHA的KV Cache大小: `[2, bs, seq_len, n_kv_head * head_dim]`, 其中2代表K和V

如果我们存储MLA up后的矩阵K,V cache，那么与MHA存储无差别

那么我们可以存储kv down的矩阵：即 c_kv = w_kv_down @ h, 此时：

`[1, bs, seq_len, dc_kv]`

如果原来  2 * n_kv_head * head_dim = dim = 2 * 4096, 如果dc_kv  = 512

那么MLA KV-cache就为MHA的 1 / 16

### MLA压缩的本质是什么？

计算时间换空间

存储时刻，w_k_up

decoding时刻 即将：

`K = w_k_up @ c_kv`

`V = w_v_up @ c_kv`

压缩KV-Cache的量的意义？以vLLM来说，减少KV-Cache量。推理服务能跑更大的batch-size，从而提高inference，decoding的效率

### MLA下位置编码问题

1. 常规算法为：RoPE(W_k_up( w_k_down(h) )), 这里的RoPE没法低秩分解
2. 可以写为attention： `Rk @ Wkup @ wkdown(h)^T` `@` `[Rq @ Wqup @ wqdown(h)]^T` 
3. 上式注意到，我们所存储kv cache是wkdown(h)为 `seq x dc_kv`, 那么我们每次计算出了要up，而且还要对**所有token**增加RoPE操作
4. 有什么方式可以减少RoPE操作吗？解决方案为在h上，增加额外的参数矩阵比如：

In [14]:
class MultiHeadsLatentAttention_withRoPE(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.n_heads = args.n_heads
        self.dim = args.dim
        # self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
        self.head_dim = args.dim // args.n_heads # 18/6 = 3
        self.dc_kv = args.dc_kv
        self.dc_q = args.dc_q

        # Q
        self.wq_down = nn.Linear(self.dim, args.dc_q, bias=False,)
        self.wq_up = nn.Linear(args.dc_q, self.dim , bias=False,)

        # 单个 C 映射到 K、V
        self.wkv_down = nn.Linear(self.dim, args.dc_kv, bias=False,)
        self.wk_up = nn.Linear(args.dc_kv, self.dim, bias=False,)
        self.wv_up = nn.Linear(args.dc_kv, self.dim, bias=False,)
        
        self.wo = nn.Linear(self.dim, self.dim, bias=False,)

        # RoPE Weight
        # K 每头一样， Q每头不一样
        self.wq_up_rope = nn.Linear(self.dc_q, self.dim , bias=False,)
        self.wk_head_rope = nn.Linear(self.dim, self.head_dim , bias=False,)

In [15]:
mla_rope = MultiHeadsLatentAttention_withRoPE(config)
print(mla_rope)

MultiHeadsLatentAttention_withRoPE(
  (wq_down): Linear(in_features=64, out_features=4, bias=False)
  (wq_up): Linear(in_features=4, out_features=64, bias=False)
  (wkv_down): Linear(in_features=64, out_features=4, bias=False)
  (wk_up): Linear(in_features=4, out_features=64, bias=False)
  (wv_up): Linear(in_features=4, out_features=64, bias=False)
  (wo): Linear(in_features=64, out_features=64, bias=False)
  (wq_up_rope): Linear(in_features=4, out_features=64, bias=False)
  (wk_head_rope): Linear(in_features=64, out_features=8, bias=False)
)


In [16]:
# 增加rope
C_Q = mla_rope.wq_down(X)
Q = mla_rope.wq_up(C_Q)
print(Q.shape)

C_KV = mla_rope.wkv_down(X)
K = mla_rope.wk_up(C_KV)
V = mla_rope.wv_up(C_KV)


# 位置编码相关
R_Q = mla_rope.wq_up_rope(C_Q) #多头
R_K = mla_rope.wk_head_rope(X) #单头
print(R_Q.shape)
print(R_K.shape)

torch.Size([3, 5, 64])
torch.Size([3, 5, 64])
torch.Size([3, 5, 8])


产生新的疑问，r_q 和 r_k 维度不同，那么做rope的dim如何处理？

In [17]:
# 简易实现 RoPE

R_Q = R_Q.view(bs, seq_len, mla_rope.n_heads, mla_rope.head_dim).transpose(1,2)
R_K = R_K.unsqueeze(dim = 1).repeat( repeats = [1, mla_rope.n_heads, 1, 1])
print(R_Q.shape)
print(R_K.shape)

## 嵌入rope
rope_matrix_q = [torch.randn(mla_rope.head_dim, mla_rope.head_dim)] * seq_len
rope_matrix_k = [torch.randn(mla_rope.head_dim, mla_rope.head_dim)] * seq_len
def apply_rope_q(X, seq_len, rope_matrix):
    for i in range(seq_len):
        X[:, i, :] = X[:, i, :] @ rope_matrix[i]
    return X
    
RoPE_Q = apply_rope_q(R_Q, seq_len, rope_matrix_q)
RoPE_K = apply_rope_q(R_K, seq_len, rope_matrix_k)

torch.Size([3, 8, 5, 8])
torch.Size([3, 8, 5, 8])


对于q每头，cat不一样的位置信息

对于k每头，cat一样的信息

In [18]:
# 与传统多头注意力无差别
# 未
Q = Q.view(bs, seq_len, mla_rope.n_heads, mla_rope.head_dim).transpose(1,2)
K = K.view(bs, seq_len, mla_rope.n_heads, mla_rope.head_dim).transpose(1,2)
V = V.view(bs, seq_len, mla_rope.n_heads, mla_rope.head_dim).transpose(1,2)

print(Q.shape) 
print(K.shape)

# cat 操作
Q = torch.cat((Q, RoPE_Q), dim = -1) # Q 也称为是 Q_nope
K = torch.cat((K, RoPE_K), dim = -1)

print(Q.shape) 
print(K.shape)

torch.Size([3, 8, 5, 8])
torch.Size([3, 8, 5, 8])


In [19]:
### 常规attention
# scores = query @ key.transpose(2,3) # keys: bs, n_heads, [head_dim, seq_len]
# keys: bs, n_heads, [2head_dim, seq_len]

S = Q @ K.transpose(2, 3) / math.sqrt(2 * mla_rope.head_dim) # cat后dim维度变化，底数也有变化
P = F.softmax(S.float(), dim=-1)
Z = P @ V 
Z = Z.transpose(1, 2).contiguous().view(bs, seq_len, -1)
O = mla.wo(Z)
print(O.shape)

torch.Size([3, 5, 64])


## 增加问题

1. 为什么位置编码要分离
2. q和k的位置编码维度不一样，那么rope的维度是否一样
3. v_up 如何被 wo 吸收
5. 写出带kv_cache版本的MLA，及decoding代码
6. MLA训练的显存计算
7. MLA并行参数分配、张量并行细节和通信

### 答案1

1. 位置编码敏感
2. 不分离的话，inference阶段要对kv做重复rope变换，
3. 如果分离那么直接拼接算好的rope k(我理解其实位置编码rope k也应该是cache起来的）

原论文

> To be specific, RoPE is position-sensitive for both keys and queries

> As a result, we must recompute the keys for all the prefix tokens during inference, which will significantly hinder the inference efficiency

> During inference, the decoupled key should also be cached.

### 答案2:

>  Therefore, DeepSeek-V2 requires a total KV cache containing $(𝑑_𝑐 + 𝑑^𝑅_ℎ)𝑙 $ elements.
1. 先说KV cache量是多少, l是长度，d_c是latent维度，$d^R_h$是单头的向量维度

2. 为什么QK位置编码有区别，Q是多头$d^R_h * h$, h是头数，那么多头保精度

3. rope K是单头，从cache角度来看，存单头k rope是经济的，随着上下文长度增加，如果是k rope多头维度本质上与常规KV无差别


### 答案3

原论文公式

$$
\begin{align}
   [\mathbf{v}_{t, 1}^{C};\mathbf{v}_{t, 2}^{C};...;\mathbf{v}_{t, n_{h}}^{C}] = \mathbf{v}_{t}^{C} &= W^{UV} \mathbf{c}_{t}^{KV}, \\
    \mathbf{o}_{t, i} &= \sum_{j=1}^{t} \operatorname{Softmax}_j(\frac{\mathbf{q}_{t, i}^T \mathbf{k}_{j, i}}{\sqrt{d_{h} + d_{h}^{R}}}) \mathbf{v}_{j, i}^{C}, \\
    \mathbf{u}_{t} &= W^{O} [\mathbf{o}_{t, 1};\mathbf{o}_{t, 2};...;\mathbf{o}_{t, n_{h}}],
\end{align}
$$

关于吸收, 原论文提到uv吸收到wo

> Fortunately, due to the associative law of matrix multiplication, we can absorb $W^{UK}$ into $W^{UQ}$, and $W^{UV}$ into $W^{O}$. 

In [20]:
# 编程举例 WUV 到 WO的矩阵吸收

c = 64
d = 4096
seq_len = 16 # length

c_kv = torch.randn(seq_len, c)
W_O = torch.randn(d, d)
W_UV = torch.randn(c, d)

Q = torch.randn(seq_len, d) # ignore rope q
K = torch.randn(seq_len, d) # ignore rope k

# 原论文公式代码，忽略底数 
V = c_kv @ W_UV
S = Q @ K.t() 
P = F.softmax(S, dim = -1)
O = P @ V
U = O @ W_O
print(U[:5,:5])

# 吸收版本
w_uv_absord = W_UV @ W_O  
# 那么实际部署时，就保留这个参数矩阵 w_uv_absord。
# 原本存 dxd + cxd, 那么现在只存 cxd (减少的参数量体感非常明显)

V = c_kv @ w_uv_absord
S = Q @ K.t()  
P = F.softmax(S, dim = -1)
O = P @ V
# U = O @ W_o # 不需要再变换了
print(O[:5,:5])

tensor([[-701.2956, -410.6637, -586.3719,  204.6843,   56.4123],
        [-258.3982, -733.2527, -224.1205,  125.4380,   -6.8913],
        [-258.3982, -733.2527, -224.1205,  125.4380,   -6.8913],
        [ 468.6087,  -20.3747, 1123.2045, -544.6324,  241.1628],
        [-264.7623, -486.6645, -184.1303,  127.2174,  143.6075]])
tensor([[-701.2963, -410.6649, -586.3724,  204.6843,   56.4113],
        [-258.3976, -733.2532, -224.1204,  125.4391,   -6.8913],
        [-258.3976, -733.2532, -224.1204,  125.4391,   -6.8913],
        [ 468.6092,  -20.3742, 1123.2051, -544.6312,  241.1630],
        [-264.7619, -486.6647, -184.1305,  127.2183,  143.6076]])


### 答案4

1. 分析有哪些矩阵可以吸收

> Fortunately, due to the associative law of matrix multiplication, we can absorb $W^{UK}$ into $W^{UQ}$, and $W^{UV}$ into $W^{O}$.

2. 另外q的矩阵可以吸收，q矩阵除了常规的 wq x = Q 之外， 还得做一次 wq_rope x = q_rope, 增加一次矩阵乘

3. kv cache要存 1. `rope k`， 2.latent `C`


### 答案5复杂，先skip

### 答案6分析

#### 切分方案A

1. 先说down 参数矩阵， Wk_down ( d x c ), 如果按照c切分，每个GPU 存储 wk_down_shard (d x c'), 输出 h(l,d) -> C(l, c')

2. up 矩阵，wk_up (c x d), 按照c切分, 每个gpu存储 wk_up_shard(c' x d) ，每个GPU输出   C(l, c') x  (c', d) -> K(l, d)

这里就有问题K在每个GPU上都是满头的， 重新切分：

#### 切分方案B 

1. 先说down 参数矩阵， Wk_down ( d x c ), 如果按照c切分，每个GPU 存储 wk_down_shard (d x c'), 输出 h(l,d) -> C(l, c')

2. 将每个 C(l, c') 分发给其他GPU， 那么每块GPU都有完整的 C(l, c)， # 考虑到c‘是非常小量的, 比如c是64， c‘是8

3. up 矩阵，wk_up (c x d), 按照d切分,  每个gpu存储 wk_up_shard(c x d'), 每个GPU输出 C(l, c) x  (c, d') -> K(l, d')

所以这样，每个头独立存在各GPU上，那么就可以head-parallel算 多头注意力了。

- 待分析， 上述未分析是否backward通信友好


依次类推可以设计出MLA的tensor parallel架构
